# Stacking and back propagation on Mnist with multiple sigmoids

### import data

In [26]:
from sklearn.datasets import fetch_mldata
import numpy as np
from sklearn.utils import shuffle
mnist = fetch_mldata('MNIST original', data_home=".")
mnist.data, mnist.target = shuffle(mnist.data, mnist.target)
X = mnist.data
s = np.ones(X.shape[0])
X_1 = np.hstack((X, s.reshape(len(s),1)))
y_float = mnist.target
y_int = y_float.astype(np.int64)
y = np.eye(10)[y_int]
N = X.shape[1]

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_1, y, test_size=0.25, random_state=42)

### define function

In [28]:
def linear_output_z1(x, w1):
    z1 = np.dot(x, w1)
    return z1

def logistic_output_z2(x, w2):
    z2 = sigmoid(np.dot(x, w2))
    return z2

def sigmoid(a):
    return 1.0 / (1.0 + np.exp(-a))

### define gradient

In [29]:
def logistic_output(x, z1, z2, theta1, theta2, theta3):
    zf = (z1 * theta1) + (z2 * theta2) + np.array(np.dot(x, theta3))
    return zf

def gradient_c_theta1(z1, zf, y):
    gradient = np.matmul(np.transpose(z1), (sigmoid(zf)-y)) / N
    return gradient

def gradient_c_theta2(z2, zf, y):
    gradient = np.matmul(np.transpose(z2), (sigmoid(zf)-y)) / N
    return gradient

def gradient_c_theta3(x, zf, y):
    gradient = np.matmul(np.transpose(x), (sigmoid(zf)-y)) / N
    return gradient

def gradient_c_w1(x, theta1, zf, y):
    gradient = np.matmul(np.transpose(x), np.transpose(np.matmul(theta1, np.transpose(np.array((sigmoid(zf) - y)))))) / N
    return gradient

def gradient_c_w2(x, w2, theta2, zf, y):
    elemsig = sigmoid(np.dot(x, w2)) # [5000, 1]
    elem1=np.transpose(np.array(sigmoid(zf) - y)) # [10,5000]
    elem2=np.matmul(theta2, elem1) # [1, 5000]
    elem3=np.matmul(np.transpose(elemsig), np.transpose(elem2)) # [1,1]
    elem4 = (elemsig* (1 - elem3)) # [5000,1]
    gradient= np.matmul(np.transpose(x), elem4)/ N # [785,1]

    return gradient


### define gradient descent 

In [43]:
def gradient_descent(x, y, int_w1, int_w2, int_theta1, int_theta2, int_theta3, lowtrain, uptrain, iterations):

    place_w1=int_w1
    place_w2=int_w2
    place_theta1=int_theta1
    place_theta2=int_theta2
    place_theta3=int_theta3

    for i in range(iterations):
        trainrate=np.random.uniform(lowtrain,uptrain,1)/np.cbrt(i+1) 
        z1 = linear_output_z1(x, place_w1)
        z2 = logistic_output_z2(x, place_w2)
        zf = logistic_output(x, z1, z2, place_theta1, place_theta2, place_theta3)
        p = sigmoid(zf)

        gradient_theta1 = gradient_c_theta1(z1, zf, y)
        gradient_theta2 = gradient_c_theta2(z2, zf, y)
        gradient_theta3 = gradient_c_theta3(x, zf, y)
        gradient_w1 = gradient_c_w1(x, place_theta1, zf, y)
        gradient_w2 = gradient_c_w2(x, place_w2, place_theta2, zf, y)
        
            
        place_w1 = place_w1 - (trainrate * gradient_w1) - (0.1/N)*place_w1 * trainrate
        place_w2 = place_w2 - (trainrate * gradient_w2) - (0.1/N)*place_w2 * trainrate
        place_theta1 = place_theta1 - trainrate*gradient_theta1*0.01 - (0.1/N)*place_theta1 * trainrate 
        place_theta2 = place_theta2 - trainrate*gradient_theta2 - (0.1/N)*place_theta2 * trainrate
        place_theta3 = place_theta3 - trainrate*gradient_theta3 - (0.1/N)*place_theta3 * trainrate

    
    return  p, place_theta1, place_theta2, place_theta3, place_w1, place_w2


### initialization

In [ ]:
N_linear = 1
M_logistic = 1

int_w1, int_w2 = np.random.uniform(-1,1,(X.shape[1]+1,N_linear)), np.random.uniform(-1,1,(X.shape[1]+1, M_logistic))
int_theta1, int_theta2, int_theta3 = np.random.uniform(-1,1,(N_linear,10)), np.random.uniform(-1,1,(M_logistic,10)),np.random.uniform(-1,1,(X.shape[1] + 1,10))
# int_w1 = result[4]
# int_w2 = result[5]
# int_theta1 = result[1]
# int_theta2 = result[2]
# int_theta3 = result[3]

result = gradient_descent(X_train, y_train, int_w1, int_w2, int_theta1, int_theta2, int_theta3, 0.0001, 0.001, 2000)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: overflow encountered in exp
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
result[0][0]

In [ ]:
def predict_accuracy(xtest, ytest, theta1, theta2, theta3, w1, w2):
    z1 = linear_output_z1(xtest, w1)
    z2 = logistic_output_z2(xtest, w2)
    zf = logistic_output(xtest, z1, z2, theta1, theta2, theta3)
    p = sigmoid(zf)
#     prediction = np.argmax(p[0], axis=1)
#     answer = np.argmax(ytest, axis=1)
#     success_rate = answer == prediction
#     success_rate = success_rate*1
#     prediction_rate = sum(success_rate) / 5000
    
    return p

In [ ]:
predict_accuracy(X_test, y_test, result[1], result[2], result[3], result[4], result[5])

In [ ]:
prediction = np.argmax(result[0], axis=1)
answer = np.argmax(y, axis=1)
success_rate = answer == prediction
success_rate = success_rate*1
sum(success_rate) / 5000